In [163]:
import pandas as pd

originaldata = pd.read_csv("../Data/Indel_curated_newmutseq.csv")
shine_del_1aa = pd.read_csv("../Data/SHINE/clinvar_gnomAD_del_1aa.txt", sep = '\t')
shine_del_2aa = pd.read_csv("../Data/SHINE/clinvar_gnomAD_del_2aa.txt", sep = '\t')
shine_ins_1aa = pd.read_csv("../Data/SHINE/clinvar_gnomAD_ins_1aa.txt", sep = '\t')
shine_ins_2aa = pd.read_csv("../Data/SHINE/clinvar_gnomAD_ins_2aa.txt", sep = '\t')

shine = pd.concat([shine_del_1aa, shine_del_2aa, shine_ins_1aa, shine_ins_2aa], ignore_index=True)
len(shine) 
shine

,ID,CHROM,POS,REF,ALT,Pathogenicity,indel,Gene,Feature,Protein.stable.ID,Symbol,Func,Protein_position,Amino_acids,gnomAD,gnomAD_popmax,gnomAD3,UKBB
0,10:97611611:TGAG:T,10,97611611,TGAG,T,Pathogenic,3,ENSG00000241935,ENST00000370646,ENSP00000359680,HOGA1,2,313,E/-,0.000465753,0.000123235,0.000269343,9.71885e-05
1,10:124405531:TAGC:T,10,124405531,TAGC,T,Pathogenic,3,ENSG00000065154,ENST00000368845,ENSP00000357838,OAT,2,184,A/-,.,.,.,.
2,19:33387464:GTCC:G,19,33387464,GTCC,G,Pathogenic,3,ENSG00000124299,ENST00000244137,ENSP00000244137,PEPD,2,453-454,ED/D,8.04531e-06,8.88952e-06,1.31416e-05,7.47604e-06
3,22:42619859:ACAT:A,22,42619859,ACAT,A,Pathogenic,3,ENSG00000100243,ENST00000352397,ENSP00000338461,CYB5R3,2,273,M/-,.,.,.,.
4,22:42619781:CGAA:C,22,42619781,CGAA,C,Pathogenic,3,ENSG00000100243,ENST00000352397,ENSP00000338461,CYB5R3,2,299,F/-,.,.,.,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5452,21:39779227:C:CGCTGCT,21,39779227,C,CGCTGCT,gnomAD_Benign,-6,ENSG00000183067,ENST00000380588,ENSP00000369962,IGSF5,2,286,R/RCC,0.00241128,0.00413725,0.00279994,0.00421913
5453,22:20566526:A:ACAGCAGCAG,22,20566526,A,ACAGCAGCAG,gnomAD_Benign,-9,ENSG00000099917,ENST00000263205,ENSP00000263205,MED15,2,250-251,-/QQQ,.,0,0.0022583,0.00060243
5454,22:27798945:T:TTGCTGC,22,27798945,T,TTGCTGC,gnomAD_Benign,-6,ENSG00000169184,ENST00000302326,ENSP00000304956,MN1,2,533,Q/QQQ,.,0,0.00387846,0.00183635
5455,22:40418743:G:GGCGGGT,22,40418743,G,GGCGGGT,gnomAD_Benign,-6,ENSG00000196588,ENST00000355630,ENSP00000347847,MRTFA,2,665,A/APA,0.00285875,0.00383206,0.00471071,0.00488114


In [164]:
#Extracting protein IDs to fetch the protein seqeunces from BioMart (one seqeunce per ID)
ProteinIDs_shine = shine['Protein.stable.ID'].drop_duplicates()
ProteinIDs_shine.to_csv('ProteinIDs_shine.csv', index=False)

protein_ids = []
protein_seqs = []
with open('../Data/SHINE/proteinIDs.fasta', 'r') as file:
    current_id = None
    current_seq = []

    for line in file:
        line = line.strip()
        if line.startswith('>'):  # If the line starts with '>', it's a new protein ID
            if current_id is not None:
                protein_ids.append(current_id)
                protein_seqs.append(''.join(current_seq))  
            current_id = line[1:]  
            current_seq = []
        else:
            current_seq.append(line)

    if current_id is not None:
        protein_ids.append(current_id)
        protein_seqs.append(''.join(current_seq))

shine_seq = pd.DataFrame({
    'ProteinID': protein_ids,
    'Sequence': protein_seqs
})


#check that I have all the sequences
len(ProteinIDs_shine), len(shine_seq['ProteinID']) #Doubled checked the missing IDs, not identified in BioMart, therefore these sequences have been updated and no longer aviliable. Revome them from shine dataset. 


shine = shine.merge(shine_seq[['ProteinID', 'Sequence']], left_on='Protein.stable.ID', right_on='ProteinID', how='left')
shine.rename(columns={'Sequence': 'wtseq'}, inplace=True)
shine.drop(columns=['ProteinID'], inplace=True)
shine


,ID,CHROM,POS,REF,ALT,Pathogenicity,indel,Gene,Feature,Protein.stable.ID,Symbol,Func,Protein_position,Amino_acids,gnomAD,gnomAD_popmax,gnomAD3,UKBB,wtseq
0,10:97611611:TGAG:T,10,97611611,TGAG,T,Pathogenic,3,ENSG00000241935,ENST00000370646,ENSP00000359680,HOGA1,2,313,E/-,0.000465753,0.000123235,0.000269343,9.71885e-05,MLGPQVWSSVRQGLSRSLSRNVGVWASGEGKKVDIAGIYPPVTTPF...
1,10:124405531:TAGC:T,10,124405531,TAGC,T,Pathogenic,3,ENSG00000065154,ENST00000368845,ENSP00000357838,OAT,2,184,A/-,.,.,.,.,MFSKLAHLQRFAVLSRGVHSSVASATSVATKKTVQGPPTSDDIFER...
2,19:33387464:GTCC:G,19,33387464,GTCC,G,Pathogenic,3,ENSG00000124299,ENST00000244137,ENSP00000244137,PEPD,2,453-454,ED/D,8.04531e-06,8.88952e-06,1.31416e-05,7.47604e-06,MAAATGPSFWLGNETLKVPLALFALNRQRLCERLRKNPAVQAGSIV...
3,22:42619859:ACAT:A,22,42619859,ACAT,A,Pathogenic,3,ENSG00000100243,ENST00000352397,ENSP00000338461,CYB5R3,2,273,M/-,.,.,.,.,MGAQLSTLGHMVLFPVWFLYSLLMKLFQRSTPAITLESPDIKYPLR...
4,22:42619781:CGAA:C,22,42619781,CGAA,C,Pathogenic,3,ENSG00000100243,ENST00000352397,ENSP00000338461,CYB5R3,2,299,F/-,.,.,.,.,MGAQLSTLGHMVLFPVWFLYSLLMKLFQRSTPAITLESPDIKYPLR...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5452,21:39779227:C:CGCTGCT,21,39779227,C,CGCTGCT,gnomAD_Benign,-6,ENSG00000183067,ENST00000380588,ENSP00000369962,IGSF5,2,286,R/RCC,0.00241128,0.00413725,0.00279994,0.00421913,MGQKERSTADTLPDLEEWKSAAGLRWWQTAVVDGSGSGNEVIEGPQ...
5453,22:20566526:A:ACAGCAGCAG,22,20566526,A,ACAGCAGCAG,gnomAD_Benign,-9,ENSG00000099917,ENST00000263205,ENSP00000263205,MED15,2,250-251,-/QQQ,.,0,0.0022583,0.00060243,MDVSGQETDWRSTAFRQKLVSQIEDAMRKAGVAHSKSSKDMESHVF...
5454,22:27798945:T:TTGCTGC,22,27798945,T,TTGCTGC,gnomAD_Benign,-6,ENSG00000169184,ENST00000302326,ENSP00000304956,MN1,2,533,Q/QQQ,.,0,0.00387846,0.00183635,MFGLDQFEPQVNSRNAGQGERNFNETGLSMNTHFKAPAFHTGGPPG...
5455,22:40418743:G:GGCGGGT,22,40418743,G,GGCGGGT,gnomAD_Benign,-6,ENSG00000196588,ENST00000355630,ENSP00000347847,MRTFA,2,665,A/APA,0.00285875,0.00383206,0.00471071,0.00488114,MDFSSVVCLPPSVIAVNGLDGGGAGENDDEPVLVSLSAAPSPQSEA...


In [165]:
#Remove indels with empty wtseq
shine = shine.dropna(subset=['wtseq'])
shine #dropped 6 indels

,ID,CHROM,POS,REF,ALT,Pathogenicity,indel,Gene,Feature,Protein.stable.ID,Symbol,Func,Protein_position,Amino_acids,gnomAD,gnomAD_popmax,gnomAD3,UKBB,wtseq
0,10:97611611:TGAG:T,10,97611611,TGAG,T,Pathogenic,3,ENSG00000241935,ENST00000370646,ENSP00000359680,HOGA1,2,313,E/-,0.000465753,0.000123235,0.000269343,9.71885e-05,MLGPQVWSSVRQGLSRSLSRNVGVWASGEGKKVDIAGIYPPVTTPF...
1,10:124405531:TAGC:T,10,124405531,TAGC,T,Pathogenic,3,ENSG00000065154,ENST00000368845,ENSP00000357838,OAT,2,184,A/-,.,.,.,.,MFSKLAHLQRFAVLSRGVHSSVASATSVATKKTVQGPPTSDDIFER...
2,19:33387464:GTCC:G,19,33387464,GTCC,G,Pathogenic,3,ENSG00000124299,ENST00000244137,ENSP00000244137,PEPD,2,453-454,ED/D,8.04531e-06,8.88952e-06,1.31416e-05,7.47604e-06,MAAATGPSFWLGNETLKVPLALFALNRQRLCERLRKNPAVQAGSIV...
3,22:42619859:ACAT:A,22,42619859,ACAT,A,Pathogenic,3,ENSG00000100243,ENST00000352397,ENSP00000338461,CYB5R3,2,273,M/-,.,.,.,.,MGAQLSTLGHMVLFPVWFLYSLLMKLFQRSTPAITLESPDIKYPLR...
4,22:42619781:CGAA:C,22,42619781,CGAA,C,Pathogenic,3,ENSG00000100243,ENST00000352397,ENSP00000338461,CYB5R3,2,299,F/-,.,.,.,.,MGAQLSTLGHMVLFPVWFLYSLLMKLFQRSTPAITLESPDIKYPLR...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5452,21:39779227:C:CGCTGCT,21,39779227,C,CGCTGCT,gnomAD_Benign,-6,ENSG00000183067,ENST00000380588,ENSP00000369962,IGSF5,2,286,R/RCC,0.00241128,0.00413725,0.00279994,0.00421913,MGQKERSTADTLPDLEEWKSAAGLRWWQTAVVDGSGSGNEVIEGPQ...
5453,22:20566526:A:ACAGCAGCAG,22,20566526,A,ACAGCAGCAG,gnomAD_Benign,-9,ENSG00000099917,ENST00000263205,ENSP00000263205,MED15,2,250-251,-/QQQ,.,0,0.0022583,0.00060243,MDVSGQETDWRSTAFRQKLVSQIEDAMRKAGVAHSKSSKDMESHVF...
5454,22:27798945:T:TTGCTGC,22,27798945,T,TTGCTGC,gnomAD_Benign,-6,ENSG00000169184,ENST00000302326,ENSP00000304956,MN1,2,533,Q/QQQ,.,0,0.00387846,0.00183635,MFGLDQFEPQVNSRNAGQGERNFNETGLSMNTHFKAPAFHTGGPPG...
5455,22:40418743:G:GGCGGGT,22,40418743,G,GGCGGGT,gnomAD_Benign,-6,ENSG00000196588,ENST00000355630,ENSP00000347847,MRTFA,2,665,A/APA,0.00285875,0.00383206,0.00471071,0.00488114,MDFSSVVCLPPSVIAVNGLDGGGAGENDDEPVLVSLSAAPSPQSEA...


In [166]:
#make mut sequences
shine['Protein_start'] = shine['Protein_position'].astype(str).apply(lambda x: x.split('-')[0])
shine['Protein_start'] = shine['Protein_start'].astype(int)
shine['wt'] = shine['Amino_acids'].astype(str).apply(lambda x: x.split('/')[0])
shine['mut'] = shine['Amino_acids'].astype(str).apply(lambda x: x.split('/')[1] if '/' in x else '')
shine['wt'] = shine['wt'].replace('-', '')
shine['mut'] = shine['mut'].replace('-', '')

shine[['indel', 'Protein_position', 'Protein_start', 'Amino_acids', 'wt', 'mut']]

print(len(shine))

# Check if there is * or ? in the data
filter = shine['wt'].str.contains(r'\*|\?', regex=True, na=False) 
filter2 = shine['mut'].str.contains(r'\*|\?', regex=True, na=False)

# Drops rows with * or ?
shine = shine[~filter]
shine = shine[~filter2]

print(len(shine)) #removed 1 row


5452
5451


/tmp/ipykernel_1609900/911969652.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shine['Protein_start'] = shine['Protein_position'].astype(str).apply(lambda x: x.split('-')[0])
/tmp/ipykernel_1609900/911969652.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shine['Protein_start'] = shine['Protein_start'].astype(int)
/tmp/ipykernel_1609900/911969652.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [167]:
def mutate(sequence, protein_start, wt, mut):
    protein_start = int(protein_start) - 1
    wt = str(wt)
    mut = str(mut)
    wt_len = len(wt)
    sequence_before_wt = sequence[:protein_start]
    sequence_after_wt = sequence[protein_start + wt_len:]
    return sequence_before_wt + mut + sequence_after_wt

# Apply the function to create mutated sequences
shine['mutseq'] = shine.apply(lambda row: mutate(row['wtseq'], row['Protein_start'], row['wt'], row['mut']), axis=1)

# Add information on indel type and length
shine['length_wt'] = shine['wtseq'].apply(len)
shine['length_mut'] = shine['mutseq'].apply(len)
shine['length_indel'] = shine.apply(lambda row: abs(row['length_mut'] - row['length_wt']), axis=1)
shine['indel_type'] = shine.apply(lambda row: 'insertion' if row['length_mut'] > row['length_wt'] else 'deletion', axis=1)
shine['Consequence'] = shine.apply(lambda row: 'insertion' if row['indel'] < 0 else 'deletion', axis=1)

shine['check'] = shine.apply(lambda row: row['indel_type'] in row['Consequence'], axis=1)
shine['check'].value_counts() #one sequence is false due to the wtseq not long enough to insert the mut. Removed this indel
shine[shine['length_indel'] == 0] #one sequence is 0 due to the wtseq not long enough to insert the mut. Removed this indel

shine = shine[shine['check'] == True]
shine = shine[shine['length_indel'] != 0]
len(shine)
shine.to_csv("Indel_shine.csv", sep=',')


In [168]:
#Add the indel type columns to origional data also
originaldata['length_wt'] = originaldata['seq'].apply(len)
originaldata['length_mut'] = originaldata['mutseq'].apply(len)
originaldata['length_indel'] = originaldata.apply(lambda row: abs(row['length_mut'] - row['length_wt']), axis=1)
originaldata['indel_type'] = originaldata.apply(lambda row: 'insertion' if row['length_mut'] > row['length_wt'] else 'deletion', axis=1)
originaldata['Consequence'] = originaldata.apply(lambda row: 'insertion' if row['Consequence_x'] == 'inframe_insertion' else 'deletion', axis=1)
originaldata['check'] = originaldata.apply(lambda row: row['indel_type'] in row['Consequence'], axis=1)
originaldata

,Unnamed: 0.1,Unnamed: 0,Id_matching,Feature_x,Consequence_x,Protein_start,Protein_stop,wt,mut,seq,...,PROVEAN,SIFT-indel,VEST-indel,VVP,length_wt,length_mut,length_indel,indel_type,check,Consequence
0,0,0,0,ENST00000379198.5,inframe_insertion,4,NaN,L,LRRAWR,MKLLRRAWRRRAALGLGTLALCGAALLYLARCAAEPGDPRAMSGRS...,...,-3.44,damaging,0.05,7.0,330,335,5,insertion,True,insertion
1,1,1,1,ENST00000379198.5,inframe_deletion,136,139.0,VLAM,V,MKLLRRAWRRRAALGLGTLALCGAALLYLARCAAEPGDPRAMSGRS...,...,-11.00,damaging,0.23,98.0,330,327,3,deletion,True,deletion
2,2,2,2,ENST00000476993.2,inframe_deletion,61,69.0,VAPLPLGTG,V,MLPWTALGLALSLRLALARSGAERGPPASAPRGDLMFLLDSSASVS...,...,-27.11,neutral,0.07,100.0,446,438,8,deletion,True,deletion
3,3,3,3,ENST00000404249.8,inframe_deletion,312,314.0,EEE,E,MGDEKDSWKVKTLDEILQEKKRRKEQEEKAEIKRLKNSDDRDSKRD...,...,-2.76,neutral,0.10,15.0,781,779,2,deletion,True,deletion
4,4,4,4,ENST00000341426.9,inframe_deletion,445,446.0,EG,G,MEMEQEKMTMNKELSPDAAAYCCSACHGDETWSYNHPIRGRAKSRS...,...,-1.90,damaging,0.29,95.0,447,446,1,deletion,True,deletion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3812,3815,3815,3959,ENST00000369550.10,inframe_deletion,36,37.0,FL,F,MADAEVIILPKKHKKKKERKSLPEEDVAEIQHAEEFLIKPESKVAK...,...,-7.29,damaging,0.98,93.0,515,514,1,deletion,True,deletion
3813,3816,3816,3960,ENST00000369550.10,inframe_deletion,470,NaN,K,NaN,MADAEVIILPKKHKKKKERKSLPEEDVAEIQHAEEFLIKPESKVAK...,...,-2.20,neutral,0.37,87.0,515,514,1,deletion,True,deletion
3814,3817,3817,3961,ENST00000369550.10,inframe_deletion,498,499.0,KK,NaN,MADAEVIILPKKHKKKKERKSLPEEDVAEIQHAEEFLIKPESKVAK...,...,-4.88,neutral,0.19,93.0,515,513,2,deletion,True,deletion
3815,3818,3818,3962,ENST00000360256.9,inframe_insertion,2136,NaN,G,AYRG,MQIELSTCFFLCLLRFCFSATRRYYLGAVELSWDYMQSDLGELPVD...,...,-10.11,damaging,0.93,100.0,2352,2355,3,insertion,True,insertion


In [169]:
shine.rename(columns={'Feature': 'Feature_x'}, inplace=True)
shine.rename(columns={'Gene': 'GeneID'}, inplace=True)
shine.rename(columns={'Pathogenicity': 'Classification'}, inplace=True)
shine.rename(columns={'Symbol': 'SYMBOL'}, inplace=True)
shine.rename(columns={'wtseq': 'seq'}, inplace=True)
shine

,ID,CHROM,POS,REF,ALT,Classification,indel,GeneID,Feature_x,Protein.stable.ID,...,Protein_start,wt,mut,mutseq,length_wt,length_mut,length_indel,indel_type,Consequence,check
0,10:97611611:TGAG:T,10,97611611,TGAG,T,Pathogenic,3,ENSG00000241935,ENST00000370646,ENSP00000359680,...,313,E,,MLGPQVWSSVRQGLSRSLSRNVGVWASGEGKKVDIAGIYPPVTTPF...,328,327,1,deletion,deletion,True
1,10:124405531:TAGC:T,10,124405531,TAGC,T,Pathogenic,3,ENSG00000065154,ENST00000368845,ENSP00000357838,...,184,A,,MFSKLAHLQRFAVLSRGVHSSVASATSVATKKTVQGPPTSDDIFER...,440,439,1,deletion,deletion,True
2,19:33387464:GTCC:G,19,33387464,GTCC,G,Pathogenic,3,ENSG00000124299,ENST00000244137,ENSP00000244137,...,453,ED,D,MAAATGPSFWLGNETLKVPLALFALNRQRLCERLRKNPAVQAGSIV...,494,493,1,deletion,deletion,True
3,22:42619859:ACAT:A,22,42619859,ACAT,A,Pathogenic,3,ENSG00000100243,ENST00000352397,ENSP00000338461,...,273,M,,MGAQLSTLGHMVLFPVWFLYSLLMKLFQRSTPAITLESPDIKYPLR...,302,301,1,deletion,deletion,True
4,22:42619781:CGAA:C,22,42619781,CGAA,C,Pathogenic,3,ENSG00000100243,ENST00000352397,ENSP00000338461,...,299,F,,MGAQLSTLGHMVLFPVWFLYSLLMKLFQRSTPAITLESPDIKYPLR...,302,301,1,deletion,deletion,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5452,21:39779227:C:CGCTGCT,21,39779227,C,CGCTGCT,gnomAD_Benign,-6,ENSG00000183067,ENST00000380588,ENSP00000369962,...,286,R,RCC,MGQKERSTADTLPDLEEWKSAAGLRWWQTAVVDGSGSGNEVIEGPQ...,408,410,2,insertion,insertion,True
5453,22:20566526:A:ACAGCAGCAG,22,20566526,A,ACAGCAGCAG,gnomAD_Benign,-9,ENSG00000099917,ENST00000263205,ENSP00000263205,...,250,,QQQ,MDVSGQETDWRSTAFRQKLVSQIEDAMRKAGVAHSKSSKDMESHVF...,789,792,3,insertion,insertion,True
5454,22:27798945:T:TTGCTGC,22,27798945,T,TTGCTGC,gnomAD_Benign,-6,ENSG00000169184,ENST00000302326,ENSP00000304956,...,533,Q,QQQ,MFGLDQFEPQVNSRNAGQGERNFNETGLSMNTHFKAPAFHTGGPPG...,1321,1323,2,insertion,insertion,True
5455,22:40418743:G:GGCGGGT,22,40418743,G,GGCGGGT,gnomAD_Benign,-6,ENSG00000196588,ENST00000355630,ENSP00000347847,...,665,A,APA,MDFSSVVCLPPSVIAVNGLDGGGAGENDDEPVLVSLSAAPSPQSEA...,1032,1034,2,insertion,insertion,True


In [170]:
import numpy as np
overlaps = shine[shine['mutseq'].isin(originaldata['mutseq'])]
overlaps

shine_unique = shine[~shine['mutseq'].isin(overlaps['mutseq'])]
shine



indel_extended = pd.concat([originaldata, shine_unique])
indel_extended['labels_cleaned'] = np.where(indel_extended['Classification'].str.contains('benign', case=False, na=False), 
                                'Benign', 
                                np.where(indel_extended['Classification'].str.contains('pathogenic', case=False, na=False), 
                                         'Pathogenic', 
                                         np.nan))
indel_extended.to_csv('Indel_extended.csv')


len(indel_extended)
indel_extended['indel_type'].value_counts()
indel_extended['labels_cleaned'].value_counts()
(indel_extended['length_indel'] == 1).value_counts()

group2to4 = indel_extended[(indel_extended['length_indel'] >= 2) & (indel_extended['length_indel'] <= 4)]
len(group2to4)
group5to10 = indel_extended[(indel_extended['length_indel'] >= 5) & (indel_extended['length_indel'] <= 10)]
len(group5to10)
(indel_extended['length_indel'] >= 11).value_counts()

#Remove duplicates, after created identical sequences. 
#combine with our dataset (if we have unique sequences)
#check the natural distribution of indels human genome

length_indel
False    6202
True      264
Name: count, dtype: int64

In [171]:
naturedata = pd.read_csv('ClinVar_indel_benchmark_with_predictions.csv')
naturedata['full_alt_seq']

naturedata['contains'] = naturedata['window_alt_seq'].apply(lambda x: any(indel_extended['mutseq'].str.contains(x, regex=False)))
naturedata['contains'].value_counts()


naturedata_unique = naturedata[naturedata['contains'] == False]


In [172]:
naturedata_unique['mut_type'].value_counts()
naturedata_unique['indel_type'] = naturedata_unique.apply(lambda row: 'insertion' if row['ref_alt_length_diff'] < 0 else 'deletion', axis=1)
naturedata_unique['indel_type'].value_counts()

/tmp/ipykernel_1609900/3731942326.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  naturedata_unique['indel_type'] = naturedata_unique.apply(lambda row: 'insertion' if row['ref_alt_length_diff'] < 0 else 'deletion', axis=1)


indel_type
deletion     689
insertion    414
Name: count, dtype: int64

In [173]:
#cleanup the naturedata column names to match our indel_extended

naturedata_unique.rename(columns={'start_pos': 'Protein_start'}, inplace=True)
naturedata_unique.rename(columns={'end_pos': 'Protein_stop'}, inplace=True)
naturedata_unique.rename(columns={'site_ref_seq': 'wt'}, inplace=True)
naturedata_unique.rename(columns={'site_alt_seq': 'mut'}, inplace=True)
naturedata_unique.rename(columns={'window_ref_seq': 'seq'}, inplace=True)
naturedata_unique.rename(columns={'window_alt_seq': 'mutseq'}, inplace=True)
naturedata_unique.rename(columns={'num_aa_affected': 'length_indel'}, inplace=True)
naturedata_unique['labels_cleaned'] = naturedata_unique.apply(lambda row: 'Pathogenic' if row['label'] == 1 else 'Benign', axis=1)

naturedata_unique['length_indel'].value_counts()

/tmp/ipykernel_1609900/2662015425.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  naturedata_unique.rename(columns={'start_pos': 'Protein_start'}, inplace=True)
/tmp/ipykernel_1609900/2662015425.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  naturedata_unique.rename(columns={'end_pos': 'Protein_stop'}, inplace=True)
/tmp/ipykernel_1609900/2662015425.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  naturedata_unique.rename(c

length_indel
1      378
2      310
3       95
4       78
6       60
5       50
7       29
9       23
8       23
10      12
13       5
12       5
11       5
21       3
14       3
18       2
32       2
22       2
20       2
26       2
16       2
145      1
24       1
197      1
58       1
39       1
207      1
27       1
15       1
66       1
204      1
152      1
59       1
Name: count, dtype: int64

In [174]:
indel_extended = pd.concat([indel_extended, naturedata_unique])
indel_extended.to_csv('Indel_extended.csv')
len(indel_extended)


7569

In [175]:
indel_extended['indel_type'].value_counts()

indel_type
deletion     5157
insertion    2412
Name: count, dtype: int64

In [176]:
indel_extended['labels_cleaned'].value_counts()

labels_cleaned
Benign        4638
Pathogenic    2931
Name: count, dtype: int64

In [177]:
(indel_extended['length_indel'] == 1).value_counts()

length_indel
False    3868
True     3701
Name: count, dtype: int64

In [178]:
group2to4 = indel_extended[(indel_extended['length_indel'] >= 2) & (indel_extended['length_indel'] <= 4)]
len(group2to4)

2512

In [179]:
group5to10 = indel_extended[(indel_extended['length_indel'] >= 5) & (indel_extended['length_indel'] <= 10)]
len(group5to10)

1047

In [180]:
(indel_extended['length_indel'] >= 11).value_counts()

length_indel
False    7260
True      309
Name: count, dtype: int64

Initial outlook on the SHINE indel dataset
Finding the 1-2 aa indels and comparing to our dataset

In [181]:
#shine['indel'].value_counts()

In [182]:
#shine_1to2aa = shine[(shine['indel'] == 3) | (shine['indel'] == -3) | (shine['indel'] == 6) | (shine['indel'] == -6) | (shine['indel'] == 9) | (shine['indel'] == -9)]
#len(shine_1to2aa)

In [183]:
#originaldata

In [184]:
#shine

In [185]:
#Check for overlapping indels in alldata
#overlapping_genes = originaldata[originaldata['GeneID'].isin(shine['Gene'])] 
#overlapping_genes2 = shine[shine['Gene'].isin(originaldata['GeneID'])]
#overlapping_genes2

In [186]:
#Removing overlapping Genes that we allready have in the original data from the shine dataset
#shine_unique = shine[~shine['Gene'].isin(overlapping_genes['GeneID'])]
#our_unique = originaldata[~originaldata['GeneID'].isin(overlapping_genes2['Gene'])]
#shine_unique_1to2aa = shine_1to2aa[~shine_1to2aa['Gene'].isin(overlapping_genes['GeneID'])]

#len(shine), len(shine_1to2aa), len(shine_unique), len(our_unique), len(shine_unique_1to2aa)